In [2]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta

def append_row(df, row):
    return pd.concat([
                df, 
                pd.DataFrame([row], columns=row.index)]
           ).reset_index(drop=True)

def getRSI14(csvfilename):
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty:
                return 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                if pd.isna(df['rsi14'].iloc[-1]):
                    return 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                return rsival, ltp
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    if symbol.endswith('.NS'):
        symbol = symbol.replace(".NS", "_NS")

    daylocationstr = f'DATASETS/Daily_Data/{symbol}.csv'
    weeklocationstr = f'DATASETS/Weekly_Data/{symbol}.csv'
    monthlocationstr = f'DATASETS/Monthly_Data/{symbol}.csv'

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp = getRSI14(daylocationstr)
    weekrsi14, wltp = getRSI14(weeklocationstr)
    monthrsi14, mltp = getRSI14(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname,
        #'ltp': dltp,
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14
    })
    return new_row

def generateGFS(scripttype):
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14'])

    fname = f'DATASETS/{scripttype}.csv'
    csvfilename = f'GFS_{scripttype}.csv'
    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.replace("\n","")
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf
df3=generateGFS('indicesdf')
df3

C:\Users\manoj\AppData\Local\Temp\ipykernel_27148\283447429.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([


,entrydate,indexcode,indexname,dayrsi14,weekrsi14,monthrsi14
0,09/01/2025,^NSEI,NIFTY 50\n,51.51,50.15,66.24
1,09/01/2025,^CNXAUTO,NIFTY AUTO\n,57.06,49.37,84.84
2,09/01/2025,^NSEBANK,NIFTY BANK\n,45.84,51.24,63.59
3,09/01/2025,^CNXFMCG,NIFTY FMCG\n,55.59,46.11,84.78
4,09/01/2025,^CNXIT,NIFTY IT\n,52.36,60.73,69.02
5,09/01/2025,^CNXMETAL,NIFTY METAL\n,35.38,42.76,71.94
6,09/01/2025,^CNXPHARMA,NIFTY PHARMA\n,0.00,0.00,0.00
7,09/01/2025,^CNXREALTY,NIFTY REALTY\n,42.86,51.89,84.86
8,09/01/2025,^CNXMEDIA,NIFTY MEDIA\n,31.89,40.62,46.95
9,09/01/2025,^CNXCONSUM,NIFTY CONSUMER GOODS,58.68,52.79,84.63


In [1]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta
from tabulate import tabulate

def append_row(df, row):
    return pd.concat([
        df,
        pd.DataFrame([row], columns=row.index)
    ]).reset_index(drop=True)

def getRSI14_and_BB(csvfilename):
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty or 'Close' not in df.columns:
                return 0.00, 0.00, 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                bb = ta.bbands(df['Close'], length=20)
                if bb is None or df['rsi14'] is None:
                    return 0.00, 0.00, 0.00, 0.00
                df['lowerband'] = bb['BBL_20_2.0']
                df['middleband'] = bb['BBM_20_2.0']
                if pd.isna(df['rsi14'].iloc[-1]) or pd.isna(df['lowerband'].iloc[-1]) or pd.isna(df['middleband'].iloc[-1]):
                    return 0.00, 0.00, 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                    lowerband = df['lowerband'].iloc[-1].round(2)
                    middleband = df['middleband'].iloc[-1].round(2)
                return rsival, ltp, lowerband, middleband
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00, 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00, 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    if symbol.endswith('.NS'):
        symbol = symbol.replace(".NS", "_NS")

    daylocationstr = f'DATASETS/Daily_Data/{symbol}.csv'
    weeklocationstr = f'DATASETS/Weekly_Data/{symbol}.csv'
    monthlocationstr = f'DATASETS/Monthly_Data/{symbol}.csv'

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp, daylowerband, daymiddleband = getRSI14_and_BB(daylocationstr)
    weekrsi14, wltp, weeklowerband, weekmiddleband = getRSI14_and_BB(weeklocationstr)
    monthrsi14, mltp, monthlowerband, monthmiddleband = getRSI14_and_BB(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname,
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14,
        'dltp': dltp,
        'daylowerband': daylowerband,
        'daymiddleband': daymiddleband,
        'weeklowerband': weeklowerband,
        'weekmiddleband': weekmiddleband,
        'monthlowerband': monthlowerband,
        'monthmiddleband': monthmiddleband
    })
    return new_row

def generateGFS(scripttype):
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14', 'dltp', 'daylowerband', 'daymiddleband', 'weeklowerband', 'weekmiddleband', 'monthlowerband', 'monthmiddleband'])

    fname = f'DATASETS/{scripttype}.csv'
    csvfilename = f'GFS_{scripttype}.csv'
    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.replace("\n", "")
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf

df3 = generateGFS('indicesdf')

df4 = df3.loc[
    (df3['monthrsi14'] >= 60.00) &
    (df3['weekrsi14'] >= 60.00) &
    df3['dayrsi14'].between(30, 70) &
    (df3['dltp'] > df3['daylowerband']) &
    (df3['dltp'] < df3['daymiddleband'])
]

df4 = df4.sort_values(by=['dayrsi14'], ascending=True)

if df4.empty:
    print("\033[1mNO INDICES QUALIFIES THE GFS CRITERIA\033[0m")
else:
    print(tabulate(df4, headers='keys', tablefmt='psql', showindex=False))


C:\Users\manoj\AppData\Local\Temp\ipykernel_11068\1073448849.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([


+-------------+-------------+-------------+------------+-------------+--------------+---------+----------------+-----------------+-----------------+------------------+------------------+-------------------+
| entrydate   | indexcode   | indexname   |   dayrsi14 |   weekrsi14 |   monthrsi14 |    dltp |   daylowerband |   daymiddleband |   weeklowerband |   weekmiddleband |   monthlowerband |   monthmiddleband |
|-------------+-------------+-------------+------------+-------------+--------------+---------+----------------+-----------------+-----------------+------------------+------------------+-------------------|
| 11/01/2025  | ^CNXIT      | NIFTY IT    |      52.36 |       60.73 |        69.02 | 44233.7 |        42984.5 |         44614.8 |           40565 |          42922.5 |          26588.4 |           36473.4 |
+-------------+-------------+-------------+------------+-------------+--------------+---------+----------------+-----------------+-----------------+------------------+-----

********EDIT FROM HERE********

Error loading files: [Errno 2] No such file or directory: 'indicesdf'


NameError: name 'indicesdf' is not defined

: 